# PSET 2 - Econometric Theory - Saverio Pietro Capra

In [1]:
# I import the Python modules (or libraries) that I'll need to solve the problems
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

In [2]:
# I read the csv of the dataset and I assign it to a variable
data = pd.read_csv("tracking.csv")

### Functions

Here below you can see some functions that I've defined in order to make the code below look neater.

$$\hat{ATE} = \frac{1}{n_{1}} \sum^{n}_{i=1} D_{i}y_{i}(1)- \frac{1}{n-n_{1}} \sum^{n}_{i=1} (1-D_{i})y_{i}(0)$$

In [3]:
# I define a function to compute the difference between the mean of the treatment and non-treatment effect

def ATE_estimator(df, col_name:str, variable_name:str):
    """
    df -> dataframe which contains the data on which we want to estimate the ATE
    col -> the column which indicates whether the unit received a treatment or not (1 if treatment, 0 if no treatment)
    variable -> the variable with respect to which you want to compute the ATE, so the variable of interest
    """
    # I calculate the mean of the treatment group
    treatment_mean = df[df[col_name]== 1][variable_name].mean()

    # I calculate the mean of the non-treatment group
    non_treatment_mean = df[df[col_name]== 0][variable_name].mean()

    # I calculate the difference between the two means, to estimate the ATE
    ATE = treatment_mean - non_treatment_mean

    # I return the ATE
    return ATE

## Exercise 1

### Part 1

Estimate the effect of tracking on students' end of first grade test scores

In [4]:
# I group the data by schoolid and remove the column which says whether students were above or below the median, since I don't need it right now
# I also calculate the mean scoreendfirstgrade for each school
grouped_data = data[["tracking", "scoreendfirstgrade", "schoolid"]].groupby(["schoolid"]).mean()

In [5]:
# Here I show a snapshot of the new dataframe I created
grouped_data.head(5)

,tracking,scoreendfirstgrade
schoolid,,
430,1.0,-0.184369
432,1.0,-0.178371
436,1.0,-0.068224
443,0.0,-0.024504
451,0.0,-0.757769


In [6]:
# Ordinary calculation of the ATE estimator, as shown in the lecture notes, approach

ATE_estimator(grouped_data, "tracking", "scoreendfirstgrade")

0.13391256466638107

In [7]:
# Regression approach
ATE_estimation = sm.OLS(grouped_data["scoreendfirstgrade"], sm.add_constant(grouped_data["tracking"])).fit().params[1]
ATE_estimation

0.13391256466638107

From this simple estimate it seems that tracking has a positive effect on end of first grade scores

### Part 2

Use a 10% level randomization inference test to assess whether the finding of question 1
is robust or whether it rests on an unappropriate asymptotic approximation. Conclude based
on the result of this randomization test.

In [8]:
print("The total numbe of schools involved in the study is",grouped_data.shape[0])

treatment_schools = round(grouped_data[grouped_data["tracking"]==1].sum()[0])
no_treatment_schools = round(grouped_data[grouped_data["tracking"]==0].count()[0])
print(f"Among these schools {treatment_schools} were assigned the treatment, and {no_treatment_schools} were not assigned the treatment")

The total numbe of schools involved in the study is 108
Among these schools 60 were assigned the treatment, and 48 were not assigned the treatment


In order to compute the 10% randomization inference test, I referenced page 151 of "Slides_RCT.pdf" where the steps to carry out a randomization inference are indicated.

Since it's a randomization inference, I work under the following assumption (sharp null hypothesis): $$H^{s}_{0}:y_{i}(1)=y_{i}(0)\; \forall i$$

In [9]:
# I define the level of the confidence interval
alpha = 10

I randomly draw a subset of 10000 possible values for $(D_{1},\dots, D_{n})$.

I estimate $\hat{ATE}_{n}$ for these values (under the sharp null).

Reference: Slide 151 - Slides_RCT

In [10]:
# Number of permutations that I will run to carry out the randomization inference
num_permutations = 1000

# Here I store the outcome of each simulation
permutation_ATEs = []

# This is a for-loop that repeats for 10000 times
for i in range(num_permutations):
    # Creates a permuted version of the DataFrame by randomly shuffling the rows without replacement
    permuted_df = grouped_data.sample(frac=1, replace=False)

    # I assign to this variable the number of units to which I will have to assign the treatment in the MC simulation 
    # n_treatment = number of schools that received treatment, which is 60
    # Note: I tried by assigning the treatment to 50% of the schools and the result is the same
    n_treatment = treatment_schools

    # I assign to the first 60 rows (number of schools that receive treatment) of the permuted dataframe the treatment
    treatment_df = permuted_df.copy().iloc[:n_treatment]
    treatment_df["tracking"] =  1

    # I leave the remaining entries untreated
    no_treatment_df = permuted_df.copy().iloc[n_treatment:]
    no_treatment_df["tracking"] = 0

    # I concatenate the treatment and non-treatment dataframe
    # I do this, so that then I can run my function to estimate the ATE
    complete_df = pd.concat([treatment_df, no_treatment_df], axis = 0)

    # I estimate the ATE by using the function I defined above
    permutation_ATE = ATE_estimator(complete_df, "tracking", "scoreendfirstgrade")

    # I append to the list permutation_t_stats my ATE estimate
    permutation_ATEs.append(permutation_ATE)

I use the $(1 - \frac{\alpha}{2})$ th quantile and $\frac{\alpha}{2}$ th quantile of the 10000 values of $\hat{ATE}_{n}$ that I've estimated.

In this way I "estimate" the true $(1 - \frac{\alpha}{2})$ th quantile and $\frac{\alpha}{2}$ th quantile of $\hat{ATE}_{n}$  across its ${n \choose n_{1}}$ possible values

Reference: Slide 151 - Slides_RCT

In [11]:
# Determine critical value, which is the 10 percentile value of the permutation_t_stats list, which has all the t_stats of the MC simulations
critical_value1 = np.percentile(permutation_ATEs, alpha/2)
critical_value2 = np.percentile(permutation_ATEs, 100-alpha/2)

print("Critical values:", (critical_value1, critical_value2))

Critical values: (-0.13527290706394515, 0.1331496285509487)


I reject the sharp null if the value taken by $\hat{ATE}_{n}$ in the original experiment is outside the range of values defined by the two quantiles computed in the previous step.

Reference: Slide 151 - Slides_RCT

In [12]:
# Compare observed statistic to critical value

# If my estimate for the ATE is outside the range of values defined by the two quantiles, I reject
# the null
if ATE_estimation < critical_value1 or ATE_estimation > critical_value2:
    print("Finding is robust at 10% level randomization inference")
else:
    print("Finding is not robust at 10% level randomization inference")

Finding is robust at 10% level randomization inference


The finding seems robust, so we can reject the null

### Part 3

One might fear that tracking benefits strong students while harming weaker ones. Assess
whether this is a legitimate concern using the dataset at hand.

In [13]:
# First of all I try to compute a simple difference in means

# I isolate only the students below the median and I compute the mean difference between treatment and non-treatment group
ATE_bottom_half = ATE_estimator(data[data["bottomhalf"]==1], "tracking", "scoreendfirstgrade")
print("The difference in means between treated and non-treated for students BELOW the median is", ATE_bottom_half)

# I isolate only the students above the median and I compute the mean difference between treatment and non-treatment group
ATE_upper_half = ATE_estimator(data[data["bottomhalf"]==0], "tracking", "scoreendfirstgrade")
print("The difference in means between treated and non-treated for students ABOVE the median is", ATE_upper_half)

The difference in means between treated and non-treated for students BELOW the median is 0.13009291684380003
The difference in means between treated and non-treated for students ABOVE the median is 0.14925006025040954


The average treatment effect seems to be positive in both cases (students below and above the mean), and with similar values.

Now I carry out an analogous estimate with a regression

In [14]:
# For each school I compute the mean scoreendfirstgrade for students below and above the median (as you can see in the dataframe below)
grouped_schools_wrtmedian = data.groupby(["schoolid","bottomhalf"]).mean().reset_index()
grouped_schools_wrtmedian = grouped_schools_wrtmedian.set_index("schoolid")

In [15]:
grouped_schools_wrtmedian.head()

,bottomhalf,tracking,scoreendfirstgrade
schoolid,,,
430,0,1.0,0.050195
430,1,1.0,-0.410246
432,0,1.0,0.180203
432,1,1.0,-0.509363
436,0,1.0,0.610353


In [19]:
# I create two separate dataframes, in the first one I isolate the average bottom half performance for each school, while in the second I isolate the average result of above median students 
grouped_schools_bottomhalf = grouped_schools_wrtmedian[grouped_schools_wrtmedian["bottomhalf"]==1]
grouped_schools_upperhalf = grouped_schools_wrtmedian[grouped_schools_wrtmedian["bottomhalf"]==0]


In [20]:
grouped_schools_bottomhalf.head()

,bottomhalf,tracking,scoreendfirstgrade
schoolid,,,
430,1,1.0,-0.410246
432,1,1.0,-0.509363
436,1,1.0,-0.777645
443,1,0.0,-0.499816
451,1,0.0,-0.989415


In [21]:
grouped_schools_upperhalf.head()

,bottomhalf,tracking,scoreendfirstgrade
schoolid,,,
430,0,1.0,0.050195
432,0,1.0,0.180203
436,0,1.0,0.610353
443,0,0.0,0.450807
451,0,0.0,-0.633036


In [22]:
# I regress the mean performance of originally below median students for each school over the dummy which indicates whether students at that school were tracked or not
# Note: in the regression I add a constant

model_bottomhalf = sm.OLS(grouped_schools_bottomhalf["scoreendfirstgrade"], sm.add_constant(grouped_schools_bottomhalf["tracking"])).fit()

In [23]:
# These are the results of the regression
# Tracking had a positive effect on the performance of below median students 
# This result is statistically significant with respect to an alpha of 10%

model_bottomhalf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     scoreendfirstgrade   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     3.429
Date:                Wed, 16 Oct 2024   Prob (F-statistic):             0.0668
Time:                        18:46:58   Log-Likelihood:                -53.205
No. Observations:                 108   AIC:                             110.4
Df Residuals:                     106   BIC:                             115.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4793      0.058     -8.308      0.000      -0.594      -0.365
tracking       0.1433      0.077      1.852      0.067      -0.010       0.297
==============================================================================
Omnibus:                        7.707   Durbin-Watson:                   1.574
Prob(Omnibus):                  0.021   Jarque-Bera (JB):                7.717
Skew:                           0.653   Prob(JB):                       0.0211
Kurtosis:                       3.097   Cond. No.                         2.77
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
# I regress the mean performance of originally above median students for each school over the dummy which indicates whether students at that school were tracked or not
# Note: in the regression I add a constant

model_upperhalf = sm.OLS(grouped_schools_upperhalf["scoreendfirstgrade"], sm.add_constant(grouped_schools_upperhalf["tracking"])).fit()

In [25]:
# These are the results of the regression
# Tracking had a positive effect on the performance of above median students 
# This result is statistically significant with respect to an alpha of 15%

model_upperhalf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     scoreendfirstgrade   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     2.518
Date:                Wed, 16 Oct 2024   Prob (F-statistic):              0.116
Time:                        18:47:04   Log-Likelihood:                -78.779
No. Observations:                 108   AIC:                             161.6
Df Residuals:                     106   BIC:                             166.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3063      0.073      4.189      0.000       0.161       0.451
tracking       0.1556      0.098      1.587      0.116      -0.039       0.350
==============================================================================
Omnibus:                        1.585   Durbin-Watson:                   1.530
Prob(Omnibus):                  0.453   Jarque-Bera (JB):                1.508
Skew:                           0.184   Prob(JB):                        0.470
Kurtosis:                       2.553   Cond. No.                         2.77
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In both cases (for students originally below or above the median), it seems that tracking has a positive effect

## Exercise 2

### Part 1

Prove the following theorem:

Theorem 0.0.1 Using draws from a uniform distribution to generate draws from other continuous distributions.

Let $F$ denote a strictly increasing cdf. If $U$ follows the uniform [0, 1] distribution, then the cdf of $F^{-1}(U)$ is $F$.

### Proof

$F$ is a strictly increasing cdf, and therefore it has an inverse, which is $F^{-1}$.

$F$ takes as input a value $x$ and returns the probability that the random variable $X$ assumes a value that is lower than $x$.

$F^{-1}$ works in the opposite way: it receives as input a number $u$ between 0 and 1, which is a probability, and returns the value $x$ such that $P(X \leq x)= u$.

We know that the CDF of $F^{-1}(U)$ can be written in the following way: 

$$G(x)=P(F^{-1}(U) \leq x)$$

I rewrite both terms inside the function as a function of $F$: 

$$G(x)=P(F(F^{-1}(U)) \leq F(x))$$
$$G(x)=P(U \leq F(x))$$

Since $U$ is uniformly distributed, then: 
$$G(x)=P(U \leq F(x)) = F(x)$$

### Part 2

In [68]:
# I define the variable obs which contains the number of observations
obs = 1000

In [69]:
# I define the variable V which has 1000 values taken randomly from a standard normal distribution
V = np.random.normal(loc=0, scale=1, size=obs)

# I define the variable y_0 which has 1000 values taken randomly from a standard normal distribution
y_0 = np.random.normal(loc=0, scale=1, size=obs)

# I define the variable y_1 as indicated in the instructions
y_1 = 0.5*y_0+0.5*V+0.2

### Part 3

Given the way we created `y_0` and `y_1` the treatment effect is heterogeneous across units, since `y_1 - y_0 = 0.5(V-y_0)+0.2`, where `V` and `y_0` are different for each unit.

### Part 4

In [70]:
# I simply compute the mean of the differences between y_1 and y_0
ATE_1000 = np.mean(y_1-y_0)

print("ATE_1000:", ATE_1000)

ATE_1000: 0.20296737587317484


### Part 5

In [71]:
# Correlation between y_0 and y_1
corr_y0y1 = np.corrcoef(y_1, y_0)
print("The correlation coefficient is", corr_y0y1[0][1])

# Variance between y_0 and y_1
var_y0y1 = np.var(y_1-y_0)
print("The variance of (y_1-y_0) is", var_y0y1)

The correlation coefficient is 0.7086312792913708
The variance of (y_1-y_0) is 0.5134004722492221


### Part 6

In [72]:
# I assign to the variable iterations the number of times the loop will have to repeat itself
iterations = 800

In [73]:
# The list below is where I will store the ATE estimates for each iteration
ATE_estimator_list0 = []
# The list below is where I will store the upper bound ATE variance estimates for each iteration
Upper_bound_ATE_variance0 = []
# The list below is where I will the outcome of the indicator function wrt to the confidence interval
# As indicated in the instructions
# The indicator that is 1 if the actual ATE is in the confidence interval and 0 otherwise
Indicator_ATEparam_inCI0 = []

# The list below is where I will store the ATE estimates for each iteration (with manual computation, non reg-coefficient)
ATE_estimator_manual0 = []
# The list below is where I will store the upper bound ATE variance estimates for each iteration, compute manually, non with heteroscedasticity
# robust standard error
Upper_bound_ATE_manual0 = []

In [74]:
# I define the for loop (800 iterations, as stated above)

for i in range(iterations):

    # I create a dataframe which stores for all units the potential outcomes, which I defined in part 2 of exercise 2
    units = pd.DataFrame({"Y0": y_0, "Y1": y_1})

    # I create a variable containing a vector of random numbers (1000, in total) taken from a uniform distribution
    r_num = np.random.uniform(0, 1, 1000)

    # I add to the dataframe units a column with the random numbers, and then I sort the dataframe
    # based on this new column of random numbers
    units.insert(2, "Random Number", r_num, False)
    units = units.sort_values(by="Random Number")

    # I create a dummy variable D equal to 1 for the first 500 observations in the sorted dataset
    D = [1]*500+[0]*500
    # I add this dummy variable as the 4th column of the dataframe
    units.insert(3, "Dummy", D)

    # I create a variable Y = (1-D)y(0)+Dy(1)
    Y = (np.ones(1000)-units["Dummy"].values)*units["Y0"].values+units["Dummy"].values*units["Y1"].values

    # I regress Y on D using the robust option
    rlr_model = sm.RLM(Y,sm.add_constant(units["Dummy"].values), M=sm.robust.norms.HuberT())
    # I extract the coefficient of the dummy variable, since it should correspond to the estimate of ATE
    ATE_estimate_D = rlr_model.fit().params[1]
    # I append to a Python list the number ATE_estimate_D
    ATE_estimator_list0.append(ATE_estimate_D)

    # I compute the estimated ATE manually
    ATE_estimator_manual = np.mean(Y[:500]-Y[500:])
    # I append the ATE_estimated manually to the respective array
    ATE_estimator_manual0.append(ATE_estimator_manual)    

    # I estimate the upper bound of the variance by extracting the heteroscedasticity robust standard errors and by squaring them to get the variance
    # I use this approach as indicated in part 8, out of simplicity
    V_ATE_plus = rlr_model.fit(cov='H1').bse[1]
    V_ATE_plus = V_ATE_plus**2
    # I append the upper bound of the variance to an array
    Upper_bound_ATE_variance0.append(V_ATE_plus)

    # Manual calculation of the upper bound of the variance of the ATE
    Upper_bound_ATE_manual = (np.var(Y[:500],ddof= 1))/len(Y[:500])+(np.var(Y[500:],ddof= 1))/len(Y[500:])
    # I append the manual estimate to the respective array
    Upper_bound_ATE_manual0.append(Upper_bound_ATE_manual)    

    # I defined the critical value of the confidence interval
    critical_value = 1.96
    # I define the lower bound of the confidence interval
    lower_bound_IC = ATE_estimate_D-critical_value*np.sqrt(V_ATE_plus)
    # I define the upper bound of the confidence interval
    upper_bound_IC = ATE_estimate_D+critical_value*np.sqrt(V_ATE_plus)

    # If statement: if the actual ATE (estimated in point 4) is in confidence interval, I append 1 to list
    # Else, I append 0 to the list
    if ATE_1000 <= upper_bound_IC and ATE_1000 >= lower_bound_IC:
        Indicator_ATEparam_inCI0.append(1)
    else:
        Indicator_ATEparam_inCI0.append(0)

In [75]:
# I compute the mean of ATE estimates
mean_ATE_estimate = np.mean(ATE_estimator_list0)

print("Mean ATE estimate", mean_ATE_estimate)
print("Difference between average ATE estimate and actual ATE", ATE_1000-mean_ATE_estimate)

# I compute the variance of ATE estimates
variance_ATE_estimate = np.var(ATE_estimator_list0)
print("Variance of ATE estimates", variance_ATE_estimate)
print("Difference between ATE estimate variance and mean upper bound variance", variance_ATE_estimate-np.mean(Upper_bound_ATE_variance0))

# I compute the mean of the CI indicator function results
average_indicator_CI = np.mean(Indicator_ATEparam_inCI0)
print("Mean Indicator CI", average_indicator_CI)

Mean ATE estimate 0.2068719047658548
Difference between average ATE estimate and actual ATE -0.003904528892679948
Variance of ATE estimates 0.0024672024817506957
Difference between ATE estimate variance and mean upper bound variance -0.0005394522661341544
Mean Indicator CI 0.975


In [76]:
print("Mean ATE estimator manually calculated",np.mean(ATE_estimator_manual0))
print("Mean Upper Bound Var ATE Estimator manually calculated", np.mean(Upper_bound_ATE_manual0))

print("\nMean ATE estimator calculated with rreg",np.mean(ATE_estimator_list0))
print("Mean Upper Bound Var ATE Estimator calculated with rreg", np.mean(Upper_bound_ATE_variance0))

print("\nThe results are roughly the same")


Mean ATE estimator manually calculated 0.2059609948225195
Mean Upper Bound Var ATE Estimator manually calculated 0.0031154704866156622

Mean ATE estimator calculated with rreg 0.2068719047658548
Mean Upper Bound Var ATE Estimator calculated with rreg 0.00300665474788485

The results are roughly the same


Since we're working in a finite population perspective, and by the design of the loop the RAT Assumption is satisfied, then the ATE estimator is unbiased, and this is empirically confirmed by the fact that the mean ATE estimate is very close to the actual ATE.

(Slide 77 - slides_RCT) - Theorem: $\hat{ATE}_{n}$ is an unbiased estimator of $ATE_{n}$

As stated in point 3, here the treatment effect is heterogeneous, and therefore the upper bound of the ATE variance is not an unbiased estimator of the variance, but it tends to overestimate the parameter. As a consequence, it makes sense that its average is larger than the standard deviation of the ATE estimates.

(Slide 118 - slides_RCT)

As a consequence of the above mentioned results, it makes sense that over 800 iterations, more than 95% of the times the actual ATE is within the confidence interval (since the IC we're using is larger than the one we'd get with the actual standard deviation of the ATE).



### Part 7

In [77]:
# I assign to the variable iterations the number of times the loop will have to repeat itself
iterations = 800

In [78]:
# The list below is where I will store the ATE estimates for each iteration
ATE_estimator_list1 = []
# The list below is where I will store the upper bound ATE variance estimates for each iteration
Upper_bound_ATE_variance1 = []
# The list below is where I will the outcome of the indicator function wrt to the confidence interval
# As indicated in the instructions
# The indicator that is 1 if the actual ATE is in the confidence interval and 0 otherwise
Indicator_ATEparam_inCI1 = []

In [79]:
# I assign to the following variable the actual ATE amount for this exercise
ATE_1000_p7 = 0.2

In [80]:
# I define the for loop (800 iterations, as stated above)

for i in range(iterations):

    # I define the variable y_0 which takes 1000 normally distributed values (it's a vector) (standard normal distr)
    y_0 = np.random.normal(size = 1000)
    
    # I define the variable V which takes 1000 normally distributed values (it's a vector)
    # The way its values are determined is of course independent from what's going on with y_0
    V = np.random.normal(size = 1000)

    # I define the varialbe y_1 according to the rules indicated in the instructions
    y_1 = 0.5*y_0+0.5*V+0.2

    # I store y_0 and y_1 for each observation in a dataframe called units
    units = pd.DataFrame({"Y0": y_0, "Y1": y_1})

    # I create a vector of length 1000 with random numbers extracted from a uniform distribution
    r_num = np.random.uniform(0, 1, 1000)

    # I add to the dataframe units a column with the random numbers, and then I sort the dataframe
    # based on this new column of random numbers
    units.insert(2, "Random Number", r_num, False)
    units = units.sort_values(by="Random Number")

    # I create a dummy variable D equal to 1 for the first 500 observations in the sorted dataset
    D = [1]*500+[0]*500
    
    # I add this dummy variable as the 4th column of the dataframe
    units.insert(3, "Dummy", D)

    # I create a variable Y = (1-D)y(0)+Dy(1)
    Y = (np.ones(1000)-units["Dummy"].values)*units["Y0"].values+units["Dummy"].values*units["Y1"].values
    
    # I manually compute the value of ATE_1000
    ATE_estimator = np.mean(Y[:500]-Y[500:])
    # I append the value to the respective array
    ATE_estimator_list1.append(ATE_estimator)

    # I manually compute the value of the upper bound of the variance of the ATE estimator
    upper_bound_variance_estimate = (np.var(Y[:500],ddof= 1))/len(Y[:500])+(np.var(Y[500:],ddof= 1))/len(Y[500:])
    # I append the estimate above to the respective array
    Upper_bound_ATE_variance1.append(upper_bound_variance_estimate)

    # I define the critical value for the confidence interval
    critical_value = 1.96

    # I compute the lower bound and the upper boiund for the confidence interval
    lower_bound_IC = ATE_estimator-critical_value*np.sqrt(upper_bound_variance_estimate)
    upper_bound_IC = ATE_estimator+critical_value*np.sqrt(upper_bound_variance_estimate)

    # I define the indicator function which returns 1 if 0.2 is in the confidence interval and 0 otherwise
    if ATE_1000_p7 <= upper_bound_IC and ATE_1000_p7 >= lower_bound_IC:
        Indicator_ATEparam_inCI1.append(1)
    else:
        Indicator_ATEparam_inCI1.append(0)

In [81]:
# I calculate the mean of the ATE_estimator_list1 array
mean_ATE_estimate = np.mean(ATE_estimator_list1)

print("Mean ATE estimates", mean_ATE_estimate)
print("Difference between average ATE estimate and actual ATE", 0.2-mean_ATE_estimate)

# I calculate the variance of the ATE_estimator_list1 array
variance_ATE_estimate = np.var(ATE_estimator_list1)
print("Variance of ATE estimates", variance_ATE_estimate)
print("Difference between ATE estimate variance and mean upper bound variance", variance_ATE_estimate-np.mean(Upper_bound_ATE_variance1))

# I calculate the mean of the average_indicator_CI array, which contains the outcomes of the
# indicator function 
average_indicator_CI = np.mean(Indicator_ATEparam_inCI1)
print("Mean Indicator CI", average_indicator_CI)

Mean ATE estimates 0.19887301511237152
Difference between average ATE estimate and actual ATE 0.001126984887628496
Variance of ATE estimates 0.0030577796077261767
Difference between ATE estimate variance and mean upper bound variance 6.147597658202279e-05
Mean Indicator CI 0.9525


In this case, by constantly randomizing the potential outcomes in each iteration of the loop, we're simulating sampling from an infinite population. By the way the code is written, we're also fulfilling the CRAT assumption.

Since, even in this case, the ATE estimator is unbiased, the fact that the mean of the estimations is very close to the actual ATE parameter is in line with expectations (not a pun).

Reference: Theorem: $\hat{ATE}_{n}$  is an unbiased estimator of $ATE$ (page 193 - slides_RCT)

Furthermore, differently from the previous part (Part 7), under the assumptions we're using now we can reliably estimate the variance with an unbiased estimator (Theorem: the variance of $\hat{ATE}_{n}$  page 200 - slides_RCT) and this justifies the effect that the mean of the variances that we've estimated is very close to the variance of the ATE estimates.

In this case, the confidence interval is more precise than the one of the previous exercise, and therefore the fact that the actual ATE is inside the interval around 95% of the times is in line with the theory.

### Part 8

In [82]:
# I assign to the variable iterations the number of times the loop will have to repeat itself
iterations = 800

In [83]:
# The list below is where I will store the ATE estimates for each iteration
ATE_estimator_list2 = []
# The list below is where I will the outcome of the indicator function wrt to the confidence interval
Upper_bound_ATE_variance2 = []
# As indicated in the instructions
# The indicator that is 1 if the actual ATE is in the confidence interval and 0 otherwise
Indicator_ATEparam_inCI2 = []

In [84]:
# I define the for loop (800 iterations, as stated above)

for i in range(iterations):

    # I define the variable y_0 which takes 1000 normally distributed values (it's a vector) (standard normal distr)
    y_0 = np.random.normal(size = 1000)

    # I define the varialbe y_1 according to the rules indicated in the instructions
    y_1 = y_0+0.1771

    # I store y_0 and y_1 for each observation in a dataframe called units
    units = pd.DataFrame({"Y0": y_0, "Y1": y_1})
    # I create a vector of length 1000 with random numbers extracted from a uniform distribution
    r_num = np.random.uniform(0, 1, 1000)

    # I add to the dataframe units a column with the random numbers, and then I sort the dataframe
    # based on this new column of random numbers
    units.insert(2, "Random Number", r_num, False)
    units = units.sort_values(by="Random Number")

    # I create a dummy variable D equal to 1 for the first 500 observations in the sorted dataset
    D = [1]*500+[0]*500
    # I add this dummy variable as the 4th column of the dataframe  
    units.insert(3, "Dummy", D)

    # I create a variable Y = (1-D)y(0)+Dy(1)
    Y = (np.ones(1000)-units["Dummy"].values)*units["Y0"].values+units["Dummy"].values*units["Y1"].values

    # I regress Y on D using the robust option
    rlr_model = sm.RLM(Y,sm.add_constant(units["Dummy"].values), M=sm.robust.norms.HuberT())
    # I estract the coefficient of the dummy variable, since it should correspond to the estimate of ATE
    ATE_estimate_D = rlr_model.fit().params[1]
    # I append to a Python list the number ATE_estimate_D
    ATE_estimator_list2.append(ATE_estimate_D)

    # I estimate the upper bound of the variance
    V_ATE_plus = rlr_model.fit(cov='H1').bse[1]
    V_ATE_plus = V_ATE_plus**2
    # I append the upper bound of the variance to an array
    Upper_bound_ATE_variance2.append(V_ATE_plus)

    # I define the critical value for the confidence interval
    critical_value = 1.96

    # I define the t-value using the formula in step 7 of the instructions
    t_value = np.abs(ATE_estimate_D/np.sqrt(V_ATE_plus))

    # I simulate the indicator function of step 7 and store the result in an array
    if t_value > critical_value:
        Indicator_ATEparam_inCI2.append(1)
    else:
        Indicator_ATEparam_inCI2.append(0)

    

In [85]:
# I compute the mean of the results of the indicator function and print it
average_indicator_CI = np.mean(Indicator_ATEparam_inCI2)
print("Mean CI Indicator Result", average_indicator_CI)

Mean CI Indicator Result 0.78


Here, according to the way we defined the variable `y_0` (a variable coming from the standard normal), its variance is 1, and since `y_1 = y_0+0.1771`, where 0.1771 is a constant, then it has the same variance as `y_0`, which is 1. 

Considering that we're simulating sampling from an infinite population, and especially that in this case the traatment effect is homogeneous (0.1771), $\hat{V}(\hat{ATE}_{n})=\frac{V(Y(0))}{n-n_{1}}+\frac{V(Y(1))}{n_{1}}$.

So, with reference to the MDD calculation, $\frac{0.1771}{\sqrt{0.004}}\approx 2.80=(q_{1-\frac{\alpha}{2}}+q_{\lambda})$.

Since we've using a critical value of 1.96, in the exercise we assumed a level of 5% in a double-tailed test, and therefore, under this conditions, the power of the test is: $$\phi(2.80-q_{1-\frac{0.05}{2}})=\phi(q_{\lambda})$$

$$=\phi(2.80-1.96)=0.80= \lambda$$

Hence, it makes sense that mean of the indicator functions results are around 80%.

### Point 9

#### Instructions
Assume you want to use a randomized experiment to measure the effect of a treatment.

Your experiment will have 1000 subjects. 500 will be treated, while 500 will remain untreated. Moreover, given the nature of the treatment, you think it makes sense to assume that $V (Y (0))$ = $V (Y (1))$. 

No one has ever measured the effect of the specific treatment you are interested in, but a literature review of the effects of treatments with a similar cost shows that they typically produce effects in the range of 10% of a standard deviation of the outcome.

Should you embark in this experiment?

Hint: revise the lecture notes on MDD (end of first
chapter) to refresh your memory on how to proceed when the MDD is expressed as standard
deviations of the outcome.

### Answer

Working with the same data that we used for **Part 8** and under the same setting, we know that the an effect in the range of 10% of a standard deviation of the outcome would be below the MDD that we'd get with a power of 0.8 and an alpha of 0.05. Indeed, as we saw in the previous exercise, that result is 0.1771, which corresponds to 17.71%.

Therefore, we wouldn't be able to estimate that effect with enough power, so it would probably not be advisable to embark in this experiment. 

In [11]:
# I assign to a variable the value of the expected effect
effect = 0.1
# I assign to a variable the value of the variance determined in part 8
variance = 0.004

# I calculate the ratio between effect and standard deviations to get
# the sum of the two critical values, for the alpha and the power
total_q = effect/np.sqrt(variance)

# I assign the alpha to a variable
alpha = 0.05

# I subtract to the sum of the two critical values the critical value of the alpha to extract the 
# one of the power
power_q = total_q- stats.norm.ppf(1-alpha)
# I extract the power in percentage with the CDF of the standard normal
power_percent = stats.norm.cdf(power_q)
print("Power", power_percent)
print("The power we get in this case is quite low, and therefore it's better not to embark in the experiemnt")

Power 0.47459866124544475
The power we get in this case is quite low, and therefore it's better not to embark in the experiemnt


### Point 10

In [93]:
# I assign to the variable iterations the number of times the loop will have to repeat itself
iterations = 800

In [94]:
# In this array I will store the outcomes of the treatment effect estimates
ATE_estimator_list3 = []
# In this list I will store the outcomes of the indicator function that checks the outcome
# of the randomization inference test
Indicator_ATEparam_inCI3 = []

In [95]:
# I define the for loop (800 iterations, as stated above)

for i in range(iterations):
    # I define the variable y_0 which takes 10 normally distributed values (it's a vector) (standard normal distr)
    y_0 = np.random.normal(size = 10)
    # I define y_1 as equal to y_0 since we're simulating an experiment where the sharp null is 
    # actually true
    y_1 = y_0

    # I store y_0 and y_1 for each observation in a dataframe called units
    units = pd.DataFrame({"Y0": y_0, "Y1": y_1})
    # I create a vector of length 10 with random numbers extracted from a uniform distribution
    r_num = np.random.uniform(0, 1, 10)

    # I add to the dataframe units a column with the random numbers, and then I sort the dataframe
    # based on this new column of random numbers
    units.insert(2, "Random Number", r_num, False)
    units = units.sort_values(by="Random Number")

    # I create a dummy variable D equal to 1 for the first 5 observations in the sorted dataset
    D = [1]*5+[0]*5
    # I add this dummy variable as the 4th column of the dataframe  
    units.insert(3, "Dummy", D)
    # I create a variable Y = (1-D)y(0)+Dy(1)
    Y = (np.ones(10)-units["Dummy"].values)*units["Y0"].values+units["Dummy"].values*units["Y1"].values
    # I regress Y on D using the OLS
    reg = sm.OLS(Y, sm.add_constant(D)).fit()
    # I estract the coefficient of the dummy variable, since it should correspond to the estimate of ATE
    ATE_1000_estimate = reg.params[1]
    # I append to a Python list the number ATE_estimate_D
    ATE_estimator_list3.append(ATE_1000_estimate)
    # I define the array where I'll store the coefficients of the randomization inference test
    Reg_coefficients = []

    # I start the for loop with 100 repetitions
    for j in range(100):
        # I create a variable that's a copy of y_0
        y_0_copy = y_0.copy()
        # I create a variable that's a copy of y_1
        y_1_copy = y_1.copy()
        # I store y_0 and y_1 for each observation in a dataframe called units
        units1 = pd.DataFrame({"Y0": y_0_copy, "Y1": y_1_copy})
        # I create a vector of length 10 with random numbers extracted from a uniform distribution
        r_num1 = np.random.uniform(0, 1, 10)

        # I add to the dataframe units1 a column with the random numbers, and then I sort the dataframe
        # based on this new column of random numbers
        units1.insert(2, "Random Number", r_num1, False)
        units1 = units1.sort_values(by="Random Number")

        # I create a dummy variable D equal to 1 for the first 5 observations in the sorted dataset
        D_tilde = [1]*5+[0]*5
        # I add this dummy variable as the 4th column of the dataframe
        units1.insert(3, "Dummy", D_tilde)
        # I create a variable Y = (1-D)y(0)+Dy(1)
        Y = (np.ones(10)-units1["Dummy"].values)*units1["Y0"].values+units1["Dummy"].values*units["Y1"].values

        # I regress Y on D using the OLS (note that I awlays add a constant when I regress, as you can see)
        reg1 = sm.OLS(Y,sm.add_constant(D_tilde)).fit()
        # I append the coefficient of the dummy to the matrix Reg_coefficients that I've defined above
        Reg_coefficients.append(reg1.params[1])

    # I extract the 5th percentile value amount the coefficients I found in the randomization above 
    Percentile_5_regcoeff = np.percentile(Reg_coefficients,5)
    # I extract the 95th percentile value amount the coefficients I found in the randomization above
    Percentile_95_regcoeff = np.percentile(Reg_coefficients,95)

    # I simulate the indicator function. If my first ATE estimate is between the 5th and 95th percentile
    # of the coefficients introduced during the randomization I return 0, else 1
    if ATE_1000_estimate >= Percentile_5_regcoeff and ATE_1000_estimate <= Percentile_95_regcoeff:
        Indicator_ATEparam_inCI3.append(0)
    else:
        Indicator_ATEparam_inCI3.append(1)

In [96]:
# I compute the mean of the results of the indicator function and print it
print(np.mean(Indicator_ATEparam_inCI3))

0.09875


In this exercise we're simulating several randomization inference tests in a situation in which, by design, the sharp null we're testing against is actually true.

In this exercise we're using for each randomization inference test a confidence interval with a level of 10%, and therefore, it's coherent with the theory the fact that over all the 800 iterations we've only rejected the null around 10% of the times.

This works even if we only worked with 10 observations, because, as indicated in slide 142, randomization inference is fit to the case in which the sample size is not going to infinity, so in the non asymptotic case.

### Exercise 3

### Part 1

$$ATE \equiv E(Y(1)-Y(0))$$
$$= E(Y(1))-E(Y(0)) \text{ (by linearity of expectation)}$$

Since
$$E(Y(1)) = E(Y(1)|D=1)P(D=1)+E(Y(1)|D=0)P(D=0)$$
$$E(Y(0)) = E(Y(0)|D=1)P(D=1)+E(Y(0)|D=0)P(D=0)$$

Then
$$ATE \equiv E(Y(1)-Y(0)) =  E(Y(1))-E(Y(0))$$
$$=E(Y(1)|D=1)P(D=1)+E(Y(1)|D=0)P(D=0)-E(Y(0)|D=1)P(D=1)-E(Y(0)|D=0)P(D=0)$$

**Terms that are observable**
- $E(Y(1)|D=1)P(D=1)$ (So both $E(Y(1)|D=1)$ and $P(D=1)$)
- $E(Y(0)|D=0)P(D=0)$ (So both $E(Y(0)|D=0)$ and $P(D=0)$)

**Terms that are NOT observable**
- $E(Y(0)|D=1)$
- $E(Y(1)|D=0)$

### Part 2

By **Assumption 1**, we know that $Y(0) \in [l,h], Y(1) \in [l,h]$.

Hence, we can define a lower and upper bound for the ATE by bounding the unobservable terms.

**Upper Bound:**
$$E(Y(1)|D=1)P(D=1) + hP(D=0) - lP(D=1) - E(Y(0)|D=0)P(D=0)$$

**Lower Bound:**
$$E(Y(1)|D=1)P(D=1) + lP(D=0) - hP(D=1) - E(Y(0)|D=0)P(D=0)$$

### Part 3

$Y(0),Y(1) \in [l,h]$, then $E[Y(0)],E[Y(1)] \in [l,h]$.

First of all, notice that $l-h \leq 0 \leq h-l$.

Then, 
$$l-h \leq Y(1)-Y(0) \leq h-l$$
$$l-h \leq E[Y(1)-Y(0)] \leq h-l$$
$$l-h \leq ATE \leq h-l$$

Indeed, the smallest value that the **lower bound** can attain is:
$$lP(D=1)+lP(D=0)-hP(D=1)-hP(D=0)=(l-h)(P(D=1)+P(D=0))=l-h$$

We attain this by setting:
- $E(Y(1)|D=0)=l$
- $E(Y(0)|D=1)=h$


While the greatest value that it may attain is:
$$lP(D=1)-lP(D=1)+hP(D=0)-hP(D=0)=0$$


We attain this by setting:
- $E(Y(1)|D=0)=h$
- $E(Y(0)|D=1)=l$


In the case of the **upper bound**, on the contrary, we have as smallest value:
$$hP(D=1)-lP(D=0)+lP(D=0)-hP(D=1)=0$$

We attain this by setting:
- $E(Y(1)|D=0)=l$
- $E(Y(0)|D=1)=h$

And we have as largest attainable value:
$$hP(D=1)-lP(D=0)+hP(D=0)-lP(D=1)=(h-l)(P(D=1)+P(D=0))=h-l$$

We attain this by setting:
- $E(Y(1)|D=0)=h$
- $E(Y(0)|D=1)=l$

So, in general, the bounds constructed in question 2 will always, in any case, contain 0.